### This pandas script was used to prepare the intial data set for the project.
At a high level, using the complete data set from the first project (CSV), it creates a new data set by:
     - Reducing the scope to the top 10 categories of incidents;
     - adding neighborhood information for each record; and
     - storing the results in CSV annd a sqlite database

In [2]:
import pandas as pd
from LatLong import LatLong
from pandas.core.groupby import DataError
import re
from datetime import datetime
import sqlite3

In [4]:
# Start with the 2016 data that was obtained from the first project
data_df = pd.read_csv('police_report_2016.csv')
data_df.head()

,id,year,month,day_m,day_w,time,category,meta_cat,resolution,location,lat,lng,zipcode
0,160040847,2016,1,1,Friday,0:01,SUSPICIOUS OCC,WC,NONE,"(37.71118870110826, -122.4286413796569)",37.711189,-122.428641,94112
1,160114953,2016,1,1,Friday,0:01,EMBEZZLEMENT,WC,NONE,"(37.743953743297546, -122.42143054584913)",37.743954,-122.421430,94110
2,160049708,2016,1,1,Friday,0:01,NON-CRIMINAL,OI,NONE,"(37.78829612755392, -122.4217341898383)",37.788296,-122.421734,94109
3,170273486,2016,1,1,Friday,0:01,OTHER OFFENSES,OI,NONE,"(37.76496576270516, -122.46630620635018)",37.764966,-122.466306,94122
4,170368639,2016,1,1,Friday,0:01,EMBEZZLEMENT,WC,NONE,"(37.79318750604184, -122.40056471552344)",37.793188,-122.400565,94104


In [15]:
data_df['category'].nunique()

39

In [11]:
# From project 1, we had established the list of top 10 categories of incidents.
# Using this list, reduce the 2016 data set to these categories.
categories = ['LARCENY/THEFT', 'ASSAULT', 'VANDALISM', 'WARRANTS', 'VEHICLE THEFT', 
              'SUSPICIOUS OCC', 'BURGLARY', 'DRUG/NARCOTIC', 'ROBBERY', 'FRAUD']

subset_df=data_df[data_df.category.isin(categories)]

In [14]:
subset_df['category'].nunique()

10

In [ ]:
# Using the code from lat/long finder found in github (https://github.com/dylburger/sf-lat-long-mapper)
#   compile a list of neighborhood names for each incident, using its lat/long info.
# This processing took nearly a full day to complete.
# NOTE: a small percentage (roughly 0.3%) of incidents did not map to a neighborhood; the neighborhood info for these
#   these records was set to 'BAD' for easy identifation in subsequent processing (by other team members).
locations = []
for i in range(0,len(subset_df['lat'])):
    testLatLong = LatLong(latitude=data_df['lat'][i], longitude=data_df['lng'][i])
    try:
        neighborhood = testLatLong.get_neighborhood()
        locations.append(neighborhood)
    except:
        locations.append('BAD')

In [ ]:
subset_df['neighbohood'] = locations
subset_df.to_csv('complete_data.csv', index=False)

In [24]:
# At this stage, we have a subset of data with neighborhhood infrmation.
subset_df.head()

,year,month,day_m,day_w,time,category,meta_cat,resolution,location,lat,lng,zipcode,neighborhood
0,2016,1,1,Friday,0:01,SUSPICIOUS OCC,WC,NONE,"(37.71118870110826, -122.4286413796569)",37.711189,-122.428641,94112,Crocker Amazon
1,2016,1,1,Friday,0:01,EMBEZZLEMENT,WC,NONE,"(37.743953743297546, -122.42143054584913)",37.743954,-122.421430,94110,Bernal Heights
2,2016,1,1,Friday,0:01,NON-CRIMINAL,OI,NONE,"(37.78829612755392, -122.4217341898383)",37.788296,-122.421734,94109,Lower Nob Hill
3,2016,1,1,Friday,0:01,OTHER OFFENSES,OI,NONE,"(37.76496576270516, -122.46630620635018)",37.764966,-122.466306,94122,Inner Sunset
4,2016,1,1,Friday,0:01,EMBEZZLEMENT,WC,NONE,"(37.79318750604184, -122.40056471552344)",37.793188,-122.400565,94104,Financial District


In [25]:
# Check to see how many distinct neighborhoods exist in the data set
subset_df['neighborhood'].nunique()

117

In [26]:
# At this stage, we had many more distinct neighborhoods in the data set, as compared to the list of neighborhoods
#   another team member found for SF neighborhoods polygon shape (which had 37 neighborhoods). This info was
#   provided by the SF Planning commission.
# This was due to the fact that the polygon shape data was provided for only major/large neighborhoods in SF.
# To work around this issue, I manually culled the records by mapping smaller neighborhoods to one of the larger
#   neighborhoods.  Due to the large size of data set, the following heuristic was used:
#   1. Using python code, an ordered list of 117 neighborhoods was compiled.
#   2. Top 43 neighborhoods on the list, those with more 1000 incidents, were mapped to one of the 37 major 
#      neighborhoods. This step was performed using Excel (global search/replace focused on the 'neighborhood' 
#      column).
#   3. These records were then extracted from the main data set, and stored in a new CSV called
#      complete_data_analyzed.csv. This step was perfromed using a combination of python and shell programming.
# These steps are not reflected in this file; detials can be provided upon request.

In [27]:
# The next step was to add geo spatial data to each of the incidents in the data set.
# This task was accomplished by reading neighborhood geo spatial info from a CSV which contained these data, and 
#   adding a new column to the main data set. The ultimate goal was to create a data set that could be used by
#   another member in charge of feeding these to Carto (for neighborhood level maps with detailed info).
# Note: this process resulted in the size of the data set growing by over twenty fold (to around 600MB). The number
#   of records did not change; the addition geo spatial column is what caused this huge increase.
shapes_df = pd.read_csv('sf_planning_neighborhoods.csv')
shapes_df.head()

,the_geom,cartodb_id,neighborho,created_at,updated_at
0,0106000020E6100000010000000103000000010000000C...,23,Pacific Heights,2014-12-11,2014-12-11
1,0106000020E61000000100000001030000000100000015...,29,Presidio Heights,2014-12-11,2014-12-11
2,0106000020E61000000100000001030000000100000020...,31,Glen Park,2014-12-11,2014-12-11
3,0106000020E61000000100000001030000000200000094...,15,Financial District,2014-12-11,2014-12-11
4,0106000020E61000000100000001030000000100000025...,17,Mission,2014-12-11,2014-12-11


In [28]:
# Define a function which returns geo data corresponding to the neighborhood column of the record being
#     passed in (as the input parameter row).
def mapper (row):
    idx = shapes_df.loc[shapes_df['neighborho'] == row['neighborhood']].index.tolist()[0]
    geo = shapes_df.the_geom[idx]
    return geo

In [ ]:
# Now 'apply' this function to the main data set in order to add a new column containing geo spatial info
#   for each incident.
big_df['the_geom'] = subset_df.apply(mapper, axis=1)
big_df.to_csv('incidents_with_geom.csv', index=False)

In [35]:
# Now depoit all these data into a sqlite database. First connect (create) the database.
conn = sqlite3.connect("incidents.db")

In [ ]:
big_df.to_sql('police_reports', conn, if_exists = 'append', index=False)
shapes_df.to_sql('geo_shapes', conn, if_exists = 'append', index=False)

In [36]:
# Check to make sure the tables can be read
pd.read_sql_query("select * from geo_shapes limit 10;", conn)

,the_geom,cartodb_id,neighborho,created_at,updated_at
0,0106000020E6100000010000000103000000010000000C...,23,Pacific Heights,2014-12-11,2014-12-11
1,0106000020E61000000100000001030000000100000015...,29,Presidio Heights,2014-12-11,2014-12-11
2,0106000020E61000000100000001030000000100000020...,31,Glen Park,2014-12-11,2014-12-11
3,0106000020E61000000100000001030000000200000094...,15,Financial District,2014-12-11,2014-12-11
4,0106000020E61000000100000001030000000100000025...,17,Mission,2014-12-11,2014-12-11
5,0106000020E61000000100000001030000000100000034...,1,Seacliff,2014-12-11,2014-12-11
6,0106000020E61000000100000001030000000100000032...,30,South of Market,2014-12-11,2014-12-11
7,0106000020E61000001100000001030000000400000000...,20,Marina,2014-12-11,2014-12-11
8,0106000020E610000002000000010300000001000000A9...,21,Bayview,2014-12-11,2014-12-11
9,0106000020E610000001000000010300000002000000F7...,32,Potrero Hill,2014-12-11,2014-12-11


In [ ]:
# We are finally data with data preparation/creation.